In [19]:
import numpy as np
import pandas as pd

In [20]:
conjointinputDF = pd.read_csv("conjointinput.csv", sep = ";")

The input file has a ranking of 3 different features of a TV set (TV Size, TV Type, TV Color) with 3 sizes (32",37",42"), 2 types (LCD, Plasma), and 3 colors (Black, Silver, and Gold). The ranking is for a combination of the above three features.

In [21]:
conjointinputDF

,Stimulus,Rank
0,A1B1C1,2
1,A1B1C2,3
2,A1B1C3,1
3,A1B2C1,5
4,A1B2C2,6
5,A1B2C3,4
6,A2B1C1,8
7,A2B1C2,9
8,A2B1C3,7
9,A2B2C1,11


Initially we need to introduce dummy variables for every stimulus.There are in total 9 differen stimuli, and 18 different combinations

In [22]:
conjointDummyDF = pd.DataFrame(np.zeros((18,9)), columns=["Rank","A1", "A2", "A3",
                                                    "B1","B2", 
                                                    "C1", "C2",
                                                    "C3"])

In [23]:
conjointDummyDF.Rank = conjointinputDF.Rank

for index, row in conjointinputDF.iterrows(): 
    stimuli1, stimuli2, stimuli3 = conjointinputDF["Stimulus"].ix[index][:2], \
    conjointinputDF["Stimulus"].ix[index][2:4], conjointinputDF["Stimulus"].ix[index][4:6]
    
    
    conjointDummyDF.ix[index, [stimuli1,stimuli2,stimuli3]] = 1

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [24]:
conjointDummyDF.head()

,Rank,A1,A2,A3,B1,B2,C1,C2,C3
0,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


Insert the proper Stimulus names

In [25]:
fullNames = {"Rank":"Rank", \
           "A1": "32\" (81cm)","A2": "37\" (94cm)","A3": "42\" (107cm)", \
          "B1": "Plasma", "B2":"LCD", \
           "C1":"Silver", "C2":"Black", "C3": "Golden",\
          }

conjointDummyDF.rename(columns=fullNames, inplace=True)

In [26]:
conjointDummyDF.head()

,Rank,"32"" (81cm)","37"" (94cm)","42"" (107cm)",Plasma,LCD,Silver,Black,Golden
0,2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,5,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,6,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


Estimate Main Effects with a linear regression
There are different ways for parameter estimation beside linear regression depending on what kind of rating you have.
For example using Probit or Logit is the output is not a rank but a decision (1=chose stimulus, 0 = no choice).

In [27]:
import statsmodels.api as sm

In [28]:
conjointDummyDF.columns

Index([u'Rank', u'32" (81cm)', u'37" (94cm)', u'42" (107cm)', u'Plasma',
       u'LCD', u'Silver', u'Black', u'Golden'],
      dtype='object')

In [30]:
X = conjointDummyDF[[u'32" (81cm)', u'37" (94cm)', u'42" (107cm)', u'Plasma',\
       u'LCD', u'Silver', u'Black', u'Golden']]
X = sm.add_constant(X)
Y = conjointDummyDF.Rank
linearRegression = sm.OLS(Y, X). fit()
linearRegression.summary()

/usr/local/lib/python2.7/site-packages/scipy/stats/stats.py:1327: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Rank   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.203e+30
Date:                Thu, 13 Dec 2018   Prob (F-statistic):          2.60e-180
Time:                        17:11:23   Log-Likelihood:                 569.12
No. Observations:                  18   AIC:                            -1126.
Df Residuals:                      12   BIC:                            -1121.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           4.3846   5.98e-16   7.33e+15      0.000       4.385       4.385
32" (81cm)     -4.5385   1.84e-15  -2.46e+15      0.000      -4.538      -4.538
37" (94cm)      1.4615   1.84e-15   7.92e+14      0.000       1.462       1.462
42" (107cm)     7.4615   1.84e-15   4.05e+15      0.000       7.462       7.462
Plasma          0.6923   1.33e-15    5.2e+14      0.000       0.692       0.692
LCD             3.6923   1.33e-15   2.77e+15      0.000       3.692       3.692
Silver          1.4615   1.84e-15   7.92e+14      0.000       1.462       1.462
Black           2.4615   1.84e-15   1.33e+15      0.000       2.462       2.462
Golden          0.4615   1.84e-15    2.5e+14      0.000       0.462       0.462
==============================================================================
Omnibus:                        2.545   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.280   Jarque-Bera (JB):                1.621
Skew:                          -0.507   Prob(JB):                        0.445
Kurtosis:                       1.935   Cond. No.                     1.02e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.73e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Part worth values & relative importance of the stimuli
Importance of Stimuli= Max(beta) - Min(beta) 
Relative Importance of Stimuli = Importance of Stim / Sum(Importance of all Stimuli)

In [31]:
importance = []
relative_importance = []

rangePerFeature = []

begin = "A"
tempRange = []
for stimuli in fullNames.keys():
    if stimuli[0] == begin:
        tempRange.append(linearRegression.params[fullNames[stimuli]])
    elif stimuli == "Rank":
        rangePerFeature.append(tempRange)
    else:
        rangePerFeature.append(tempRange)
        begin = stimuli[0]
        tempRange = [linearRegression.params[fullNames[stimuli]]]

In [32]:
for item in rangePerFeature:
    importance.append( max(item) - min(item))

for item in importance:
    relative_importance.append(100* round(item/sum(importance),3))

In [33]:
partworths = []

item_levels = [1,3,5,8]

for i in range(1,4):
    part_worth_range = linearRegression.params[item_levels[i-1]:item_levels[i]]
    print part_worth_range

32" (81cm)   -4.538462
37" (94cm)    1.461538
dtype: float64
42" (107cm)    7.461538
Plasma         0.692308
dtype: float64
LCD       3.692308
Silver    1.461538
Black     2.461538
dtype: float64


In [34]:
meanRank = []
for i in conjointDummyDF.columns[1:]:
    newmeanRank = conjointDummyDF["Rank"].loc[conjointDummyDF[i] == 1].mean()
    meanRank.append(newmeanRank)

    
#total Mean or, "basic utility" is used as the "zero alternative"
totalMeanRank = sum(meanRank) / len(meanRank)



partWorths = {}
for i in range(len(meanRank)):
    name = fullNames[sorted(fullNames.keys())[i]]
    partWorths[name] = meanRank[i] - totalMeanRank

In [35]:
partWorths

{'32" (81cm)': -6.0,
 '37" (94cm)': 0.0,
 '42" (107cm)': 6.0,
 'Black': 1.0,
 'Golden': -1.0,
 'LCD': 1.5,
 'Plasma': -1.5,
 'Silver': 0.0}

Summary & Results

In [36]:
print "Relative Importance of Feature:\n\nMonitor Size:",relative_importance[0], "%",\
"\nType of Monitor:", relative_importance[1], "%", "\nColor of TV:", relative_importance[2], "%\n\n"

print "--"*30

print "Importance of Feature:\n\nMonitor Size:",importance[0],\
"\nType of Monitor:", importance[1],  "\nColor of TV:", importance[2]

Relative Importance of Feature:

Monitor Size: 70.6 % 
Type of Monitor: 17.6 % 
Color of TV: 11.8 %


------------------------------------------------------------
Importance of Feature:

Monitor Size: 12.0 
Type of Monitor: 2.9999999999999987 
Color of TV: 1.9999999999999925


In [37]:
#As array that looks like X
#Must include Constant!

optBundle = [1,0,0,1,0,1,0,1,0]
print "The best possible Combination of Stimuli would have the highest rank:",\
linearRegression.predict(optBundle)[0]

The best possible Combination of Stimuli would have the highest rank: 17.999999999999993


Using the Partworths:

In [38]:
#Optimal Bundle:
#42", LCD, Black

optimalWorth = partWorths["42\" (107cm)"] + partWorths["LCD"] + partWorths["Black"]

print "Choosing the optimal Combination brings the user an additional ", optimalWorth, "'units' of utility"

Choosing the optimal Combination brings the user an additional  8.5 'units' of utility
